In [2]:
!pip3 install torch

     |████████████████████████████████| 97.5 MB 2.9 MB/s eta 0:00:01    |                                | 143 kB 1.9 MB/s eta 0:00:53     |███▍                            | 10.3 MB 4.5 MB/s eta 0:00:20     |█████▎                          | 16.2 MB 4.9 MB/s eta 0:00:17     |███████▌                        | 22.8 MB 3.2 MB/s eta 0:00:24     |████████████████████████▎       | 74.1 MB 2.3 MB/s eta 0:00:11


In [5]:
!pip3 install torchvision

     |████████████████████████████████| 387 kB 205 kB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 2.1 MB/s eta 0:00:01


In [6]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad() # Clear gradients w.r.t. parameters
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

100.1%

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


113.5%

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


100.4%

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


180.4%/usr/local/lib/python3.8/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!
Epoch [1/5], Step [100/600], Loss: 0.0963
Epoch [1/5], Step [200/600], Loss: 0.1675
Epoch [1/5], Step [300/600], Loss: 0.0803
Epoch [1/5], Step [400/600], Loss: 0.2159
Epoch [1/5], Step [500/600], Loss: 0.0741
Epoch [1/5], Step [600/600], Loss: 0.0518
Epoch [2/5], Step [100/600], Loss: 0.0631
Epoch [2/5], Step [200/600], Loss: 0.1102
Epoch [2/5], Step [300/600], Loss: 0.0216
Epoch [2/5], Step [400/600], Loss: 0.0448
Epoch [2/5], Step [500/600], Loss: 0.0505
Epoch [2/5], Step [600/600], Loss: 0.0307
Epoch [3/5], Step [100/600], Loss: 0.0437
Epoch [3/5], Step [200/600], Loss: 0.0843
Epoch [3/5], Step [300/600], Loss: 0.0279
Epoch [3/5], Step [400/600], Loss: 0.0109
Epoch [3/5], Step [500/600], Loss: 0.0099
Epoch [3/5], Step [600/600], Loss: 0.0661
Epoch [4/5], Step [100/600], Loss: 0.0321
Epoch [4/5], Step [200/600], Loss: 0.0554
Epoch [4/5], Step [300/600], Loss: 0.0136
Epoch [4/5], Ste